# Rebalance - an example

This is the analytical procedure to rebalance a portfolio. 
The computations must be performed after `Dfix` closing and before the rebalance execution in the `Droll` (usually the closing of next business day after `Dfix`).

Let's assume that we already have decided to follow a portfolio strategy with the following parameters:
1. mixture of CVaR's as a risk measure, with following parameters: 
    - confidence levels `alpha = [0.95, 0.90, 0.85]`
    - mixture coefficients `coef = [0.2, 0.3, 0.5]`
2. Sharpe optimization strategy (*i.e.* maximization of mCVaR-Sharpe ratio, `rtype='Sharpe'`)
3. Quarterly rebalanced, `freq='Q'`
4. With the length of the historical data used in the calibration of the weights of `hlength=3.25` years.

>Note: The workflow is the same if another portfolio strategy was adopted. 

We start by importing the **azapy** package as well as other useful packages.

In [1]:
import pandas as pd
import sys
sys.path.append("..")
import azapy as az

print(f"azapy version {az.version()}")

azapy version 1.2.2


### Collect the relevant historical market data

- `symb` is the list of stock symbols (portfolio components).
- `sdate` and `edate` are the start and end dates of historical time-series.
- `mktdir` is the name of the directory used to save the market data collected from the data provider *alphavantage*.
    
> Note: if the flag `force=False` then a reading from `dir=mktdir` is attempted. If it fails, then the data provider servers will be accessed. The new data will be saved to the `dir=mktdir` (for more information see the azapy.readMkT function https://azapy.readthedocs.io/en/latest/documentation.)

> Hint: if you want to overwrite existing old data in directory `mktdir` then set `force=True` - in this case a fresh reading from market data provider is made and the results are saved in `mktdir`.

In [2]:
symb = ['GLD', 'TLT', 'XLV', 'VGT', 'VHT']

sdate = "2012-01-01"
edate = "2021-07-27"

mktdir = "../MkTdata"

mktdata = az.readMkT(symb, sdate=sdate, edate=edate, file_dir=mktdir)

read GLD data from file
read TLT data from file
read XLV data from file
read VGT data from file
read VHT data from file

Request between 2012-01-01 : 2021-07-27
                    GLD         TLT         XLV         VGT         VHT
source            yahoo       yahoo       yahoo       yahoo       yahoo
force             False       False       False       False       False
save               True        True        True        True        True
file_dir     ../MkTdata  ../MkTdata  ../MkTdata  ../MkTdata  ../MkTdata
file_format         csv         csv         csv         csv         csv
api_key            None        None        None        None        None
verbose            True        True        True        True        True
error                No          No          No          No          No
nrow               2407        2407        2407        2407        2407
sdate        2012-01-03  2012-01-03  2012-01-03  2012-01-03  2012-01-03
edate        2021-07-27  2021-07-27  2021-07-27

### Check the market data

At this point it is a good idea to check the market data quality. 
The **azapy** function `summary_MktData` my come in handy. It returns:

- `symbol` the symbols included in `mktdata`. These are the portfolio components. 
- `begin` is the beginning of the historical time-series for each symbol.
- `end` is the end date of the historical time series (the most recent date).
- `length` is history length in number of business days.
- `na_total` is the total number of missing values, `nan`.
- `na_b` is the total number of missing records at the beginning of the time-series (in number of business days).
- `na_e` is the total number of missing records at the end of the time-series (in number of business days).
- `cont` is the total number of missing records (at the beginning, end and in the middle of the time-series). 

We are looking for:
- `begin` date to be prior `end` date minus `length`. In our case this condition is overwhelmingly satisfied.
- `end` must be equal or greater than `Dfix`. Therefore, it is important to retrieve the market data in the `Dfix` date after the market closing and after the data provider had updated their database. 
- `na_total`, `na_b`, `na_e` and `cont` must be `0`. Missing data could be a severe problem. In general gaps in market data should be resolved before proceeding further.

In [3]:
az.summary_MkTdata(mktdata)

,symbol,begin,end,length,na_total,na_b,na_e,cont
0,GLD,2012-01-03,2021-07-27,2407,0,0,0,0
1,TLT,2012-01-03,2021-07-27,2407,0,0,0,0
2,VGT,2012-01-03,2021-07-27,2407,0,0,0,0
3,VHT,2012-01-03,2021-07-27,2407,0,0,0,0
4,XLV,2012-01-03,2021-07-27,2407,0,0,0,0


### Set the dispersion measure parameters 

For us they are the confidence levels and mixture coefficients mentioned before.

In [4]:
alpha = [0.95, 0.90, 0.85]
coef = [0.2, 0.3, 0.5]

### Set up the CVaRAnalyzer class

In this class, `hlength` and `freq` default values are `3.25` and `'Q'`, respectively *(see the documentation)*. Here we set them explicitly for illustration purpose. 
Also we choose the maximization of C-Sharpe ration optimization strategy (rtype='Sharpe) with 0 risk free rate (mu0=0)

In [5]:
cr1 = az.CVaRAnalyzer(alpha, coef, mktdata, hlength=3.25, freq='Q', rtype='Sharpe', mu0=0)

### Set up the existing portfolio positions

- `ns` is the existing number of shares per asset. These must be greater or equal to 0. Our portfolio strategies do not allow for short positions. *See also the documentation for `getPositions` method for additional information.*
- `cash` is the additional cash in dollars that we what to add to the total investment capital. A negative value means a capital reduction (cash withdraw). In our example we retire a portion of the invested capital (*i.e.* 3000 dollars).

In [6]:
ns = pd.Series([300, 40, 150, 50, 50], index=symb)
cash = -3000

### Get the rebalancing positions

Inputs:

- `nshares` is the `pd.Series` containing the existing number of shares per asset.
- `cash` is the additional amount of cash to be added/subtracted to the capital.

Returns:
- `old_nsh` column contains the exiting number of shares per asset as well as the additional cash. In this example the capital is reduced by 3000 dollars. These are the input values.
- `new_nsh` column contains the new numbers of shares per asset. The `cash` entry is the amount of cash that must be rolled to the next rebalancing event. It is generated by rounding the number of shares to integer values. Although its value is small, it could be positive or negative. It is the investor responsibility to compensate for a capital shortfall. 
- `diff_nsh` it is the actual number of shares that need to be sold and both to rebalance the portfolio. Under the `cash` entry is the actual amount of cash that is hold by the investor after the rebalance. In our case we have asked to retire 3000 dollars. However, after rebalancing, the new position assumes a capital shortfall of 246 dollars. Therefore, we are left with only 2754 dollars.
- `weights` column holds the portfolio weights in algebraic form. Under the `cash` entry we have the total value of the shares at the closing price in the `Dfix` date. This may be different than the actual value recorder at the transaction time in the `Droll` date (usually the next business day).
- `price` the shares price at the `Difx` closing.

> Note: The rebalancing positions are computed as of `Dfix` date. However, the execution price in the `Droll` date could be bigger or smaller than the closing price in the `Difx` date. Therefore, the invested capital could be different than the one presented in this report. It is the investor responsibility to cover for the potential capital shortfall.

In [7]:
cr1.getPositions(nshares=ns, cash=cash).round(3)

Use closing prices as of 2021-07-27 00:00:00


,old_nsh,new_nsh,diff_nsh,weights,prices
GLD,300,311.0,11.0,0.493,168.44
TLT,40,51.0,11.0,0.071,149.64
VGT,50,64.0,14.0,0.248,410.31
VHT,50,0.0,-50.0,0.000,254.18
XLV,150,152.0,2.0,0.188,131.24
cash,-3000,203.3,3203.3,106224.801,1.00
